In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import os


In [2]:
gmID = 'acd71bc0-ecf4-11ee-9385-ef789ffde1d3'

In [3]:
from boto3.dynamodb.conditions import Key, Attr
import boto3

dynamodb = boto3.resource('dynamodb')
# Instantiate a table resource object
table = dynamodb.Table('ads_passenger_processed')
table_meta = dynamodb.Table('ads_passenger_processed_metadata')

In [4]:
keywords = dict(
    #IndexName='groupMetadataID-index',
    IndexName='topic-index',
    
    KeyConditionExpression=Key('groupMetadataID').eq(gmID),
    #KeyConditionExpression=Key('topic').eq(
        #'/apollo/drive_event'),
    #    '/apollo/perception/obstacles'),
    #    '/apollo/sensor/gnss/best_pose'),
    
    Limit=1500,
    
    #FilterExpression=Key('groupMetadataID').eq(f'{gmID}')
    #FilterExpression=Key('topic').eq('/apollo/drive_event')
    
    #ProjectionExpression="groupMetadataID, topic",
    
    
)

# run initial query and coerce results into a dataframe
res = table.query(**keywords)
df = pd.DataFrame.from_dict(
    pd.json_normalize(res['Items']), orient='columns')


res
done = False
leek = res["LastEvaluatedKey"]

page_num = 10

while page_num and not done:
    #page_num -= 1
    try:
        leek = res["LastEvaluatedKey"]
        keywords["ExclusiveStartKey"] = leek
        res = table.query(**keywords)
        tmpDF = pd.DataFrame.from_dict(
            pd.json_normalize(res['Items']), orient='columns')
        df = pd.concat([df, tmpDF])
    except KeyError:
        print("Done querying data")
        done = True


KeyboardInterrupt: 

In [ ]:
#df = df.sort_values(by="topic")


In [5]:
df['topic'].unique()

array(['/apollo/sensor/velodyne/fusion/PointCloud2Status'], dtype=object)

In [7]:
df

,topic,msgsize,frameId,measurementTime,channelName,time,metadataID,isDense,groupMetadataID,width,height,_id,msg_type,header.lidarTimestamp,header.sequenceNum,header.frameId,header.timestampSec
0,/apollo/sensor/velodyne/fusion/PointCloud2Status,196,,1698162328.172794,"[/apollo/sensor/velodyne32/PointCloud2, /apoll...",1698162328201639239,101ad260-eab6-11ee-b297-3b0ad9d5d6c6,True,621c07b8-eaaf-11ee-b297-3b0ad9d5d6c6,42723,1,24f9d2ba-eab6-11ee-b297-3b0ad9d5d6c6,,1698162328172794040,58277,velodyne32,1698162328.183011
1,/apollo/sensor/velodyne/fusion/PointCloud2Status,195,,1695315101.8992922,"[/apollo/sensor/velodyne32/PointCloud2, /apoll...",1695315101938897226,a629e986-ea66-11ee-b297-3b0ad9d5d6c6,True,17876fec-ea66-11ee-b297-3b0ad9d5d6c6,39872,1,da01b97c-ea66-11ee-b297-3b0ad9d5d6c6,,1695315101899292040,14145,velodyne32,1695315101.9078135
2,/apollo/sensor/velodyne/fusion/PointCloud2Status,196,,1706035078.800292,"[/apollo/sensor/velodyne32/PointCloud2, /apoll...",1706035078839713438,711569fa-f184-11ee-bab5-fb353e7798cd,True,38ac9526-f182-11ee-bab0-fb353e7798cd,44523,1,78d2aa9d-f184-11ee-bab5-fb353e7798cd,,1706035078800292040,53823,velodyne32,1706035078.8105447
3,/apollo/sensor/velodyne/fusion/PointCloud2Status,196,,1692299211.625519,"[/apollo/sensor/velodyne32/PointCloud2, /apoll...",1692299211766355167,9724aa11-db3f-11ee-a158-97f8443fd730,True,dc39aa14-db32-11ee-a158-97f8443fd730,46631,1,c041d7a1-db3f-11ee-a158-97f8443fd730,,1692299211625519040,61227,velodyne32,1692299211.690912
4,/apollo/sensor/velodyne/fusion/PointCloud2Status,196,,1699992738.467922,"[/apollo/sensor/velodyne32/PointCloud2, /apoll...",1699992738559124069,fb79b53c-eb29-11ee-b297-3b0ad9d5d6c6,True,781c0bcc-eb21-11ee-b297-3b0ad9d5d6c6,45836,1,00dd5a74-eb2a-11ee-b297-3b0ad9d5d6c6,,1699992738467922040,57411,velodyne32,1699992738.4842944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,/apollo/sensor/velodyne/fusion/PointCloud2Status,196,,1700155637.601433,"[/apollo/sensor/velodyne32/PointCloud2, /apoll...",1700155637632115610,3d8d21fe-f093-11ee-b9d0-fb353e7798cd,True,5c2ad8ec-f08c-11ee-b9c8-fb353e7798cd,41510,1,6d906478-f093-11ee-b9d0-fb353e7798cd,,1700155637601433040,43224,velodyne32,1700155637.6101468
1496,/apollo/sensor/velodyne/fusion/PointCloud2Status,196,,1702931484.5749922,"[/apollo/sensor/velodyne32/PointCloud2, /apoll...",1702931484629478452,c15c93a7-ebc9-11ee-b297-3b0ad9d5d6c6,True,a7c98b32-ebc2-11ee-b297-3b0ad9d5d6c6,42885,1,01c7191a-ebca-11ee-b297-3b0ad9d5d6c6,,1702931484574992040,96849,velodyne32,1702931484.5822942
1497,/apollo/sensor/velodyne/fusion/PointCloud2Status,196,,1690481784.324192,"[/apollo/sensor/velodyne32/PointCloud2, /apoll...",1690481784358052137,8868ad62-d9c1-11ee-a158-97f8443fd730,True,19b7ebd0-d9b7-11ee-a158-97f8443fd730,51275,1,ce6d5ec2-d9c1-11ee-a158-97f8443fd730,,1690481784324192040,115047,velodyne32,1690481784.3311174
1498,/apollo/sensor/velodyne/fusion/PointCloud2Status,196,,1698943348.782018,"[/apollo/sensor/velodyne32/PointCloud2, /apoll...",1698943348804332143,83e702a2-f091-11ee-b9ce-fb353e7798cd,True,b6227b56-f08d-11ee-b9c9-fb353e7798cd,41202,1,ab262618-f091-11ee-b9ce-fb353e7798cd,,1698943348782018040,33495,velodyne32,1698943348.787428


In [ ]:
#df.to_csv("test.csv")

In [6]:
df.shape

(931500, 17)